# Simple Example of using WebLoader with PyTorch

In [1]:
%matplotlib inline

In [2]:
from importlib import reload

import os
import numpy as np

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
from torchvision import models

import webloader as wl

from IPython import display
import matplotlib.pyplot as plt
import torchtrainers as tt

os.chdir("/tmp")

from webloader import tarrecords
print(tarrecords.errors_are_fatal)
tarrecords.errors_are_fatal = True

False


In [3]:
mean = np.array([0.485, 0.456, 0.406], "f")
std = np.array([0.229, 0.224, 0.225], "f")
def norm_image(xs):
    return (xs-mean[None,None,None,:])/std[None,None,None,:]
def norm_cls(ys):
    ys = ys.astype(np.int64)
    return ys-1

In [4]:
training_urls = "http://storage.googleapis.com/lpr-imagenet-augmented/imagenet_train-{0000..0147}-{000..019}.tgz"
training_size = 1000000
training = wl.MultiWebLoader(
    training_urls, training_size,
    fields="ppm;png;jpg cls",
    batch_transforms=[norm_image, norm_cls],
    batch_size=64,
    converters="torch",
    shuffle=1000,
    verbose=False,
    use_torch_mp=True,
    queue_size=200,
    processes=4)
wl.loader_test(training)

494.27 samples/s 7.72 batches/s
0 : Tensor torch.Size([64, 3, 224, 224]) cpu torch.float32 -2.1179039478302 2.640000104904175
1 : Tensor torch.Size([64]) cpu torch.int64 0 957


In [5]:
if os.path.exists("./imagenet_val-0000.tgz"):
    testing_urls = "./imagenet_val-0000.tgz"
else:
    testing_urls = "http://storage.googleapis.com/lpr-imagenet-augmented/imagenet_val-0000-000.tgz"
testing_size = 50000
testing = wl.WebLoader(
    testing_urls, testing_size,
    fields="ppm;png;jpg cls",
    batch_transforms=[norm_image, norm_cls],
    batch_size=64,
    epochs=1,
    converters="torch")
wl.loader_test(testing)

181.14 samples/s 2.83 batches/s
0 : Tensor torch.Size([64, 3, 224, 224]) cpu torch.float32 -2.1179039478302 2.640000104904175
1 : Tensor torch.Size([64]) cpu torch.int64 16 983


In [6]:
model = models.resnet50()
model = model.to("cuda")

In [7]:
reload(tt)
#model = nn.DataParallel(model)
criterion = nn.CrossEntropyLoss().cuda()
trainer = tt.Trainer(model=model,
                     criterion=criterion,
                     metrics=[tt.Misclassification],
                     device="cuda")
trainer.set_lr(1e-3)
trainer.fit_for(training, 5000)
print(trainer.test_for(testing, 1000))

setting optimizer: SGD ( Parameter Group 0 dampening: 0 lr: 0.001 momentum: 0 nesterov: False weight_decay: 0 )
training    5056 /     5000 time 00:00:00 / 00:00:19 101% loss   7.05573 [    79] err   0.99875
loss 7.0588 err 1.0000


In [8]:
tt.training(trainer, training, 10000, testing, 1000, epochs=10, learning_rates=[(0,1e-2), (300000, 1e-3)])

#For full training, use appropriate epoch sizes for the dataset:
#tt.training(trainer, training, 1000000, testing, 50000, epochs=100, learning_rates=[(0,1e-2), (300000, 1e-3)])

setting optimizer: SGD ( Parameter Group 0 dampening: 0 lr: 0.01 momentum: 0 nesterov: False weight_decay: 0 )
   0 test (none) ::: train  100% /00:00:37  loss 6.9733 err 0.9984       
   1 test loss 7.1559 err 0.9990 ::: train  100% /00:00:37  loss 6.9449 err 0.9988       
   2 test loss 7.0406 err 0.9971 ::: train  100% /00:00:37  loss 6.9353 err 0.9991       
   3 test loss 7.0321 err 0.9980 ::: train  100% /00:00:37  loss 6.9189 err 0.9978       
   4 test loss 6.9938 err 0.9980 ::: train  100% /00:00:37  loss 6.9250 err 0.9981       
   5 test loss 6.9262 err 0.9941 ::: train  100% /00:00:37  loss 6.8977 err 0.9981       
   6 test loss 6.9308 err 0.9971 ::: train  100% /00:00:37  loss 6.8868 err 0.9959       
   7 test loss 6.9245 err 0.9990 ::: train  100% /00:00:37  loss 6.8584 err 0.9984       
   8 test loss 6.8566 err 0.9971 ::: train  100% /00:00:37  loss 6.8452 err 0.9950       
   9 test loss 6.8744 err 0.9961 ::: train  100% /00:00:37  loss 6.8300 err 0.9959       
